In [1]:
import pandas as pd

# Load the dataset
df_sampled = pd.read_csv("C:/Users/Guest01/Documents/Manpreet_thesis/Datasets/Glassdoor_job_reviews2/all_reviews.csv")

# Downcast numerical columns to save memory
def downcast(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype == "int64"]
    df[float_cols] = df[float_cols].astype("float32")
    df[int_cols] = df[int_cols].astype("int32")
    return df

df_sampled = downcast(df_sampled)
df_sampled.head()


C:\Users\Guest01\AppData\Local\Temp\ipykernel_20140\4126841228.py:4: DtypeWarning: Columns (5,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sampled = pd.read_csv("C:/Users/Guest01/Documents/Manpreet_thesis/Datasets/Glassdoor_job_reviews2/all_reviews.csv")


,rating,title,status,pros,cons,advice,Recommend,CEO Approval,Business Outlook,Career Opportunities,Compensation and Benefits,Senior Management,Work/Life Balance,Culture & Values,Diversity & Inclusion,firm_link,date,job,index
0,5.0,Good,"Current Employee, more than 10 years",Knowledge gain of complete project,Financial growth and personal growth,NaN,v,o,v,3,3,3,3,3.0,3.0,Reviews/Baja-Steel-and-Fence-Reviews-E5462645.htm,"Nov 19, 2022",Manager Design,NaN
1,4.0,Good,"Former Employee, less than 1 year","Good work,good work , flexible, support","Good,work, flexible,good support, good team work",NaN,v,o,o,4,4,4,4,4.0,4.0,Reviews/Baja-Steel-and-Fence-Reviews-E5462645.htm,"Jan 29, 2022",Anonymous Employee,NaN
2,4.0,"Supervising the manufacturing the processes, e...","Current Employee, more than 1 year",This company is a best opportunity for me to l...,"Monthly Target work,Maintain production schedu...",NaN,v,o,v,2,3,2,2,2.0,2.0,Reviews/Baja-Steel-and-Fence-Reviews-E5462645.htm,"Aug 12, 2021",Production Engineer,NaN
3,1.0,terrible,"Current Employee, more than 1 year",I wish there were some to list,too many to list here,NaN,x,x,x,1.0,3.0,1.0,3.0,1.0,NaN,https://www.glassdoor.com/Reviews/Calgary-Flam...,"Sep 24, 2020",Senior Account Executive,NaN
4,4.0,"It could be so good, but it isn’t","Current Employee, more than 3 years",Fast Paced. Endless challenges. Inclusive envi...,The biggest perk of the job provides no value ...,NaN,o,o,o,3.0,3.0,3.0,1.0,4.0,5.0,https://www.glassdoor.com/Reviews/Calgary-Flam...,"Mar 25, 2023",Assistant Manager,NaN


In [2]:
# List of relevant features
relevant_features = [
    'pros', 'cons', 'Recommend', 'Career Opportunities', 'Compensation and Benefits', 
    'Senior Management', 'Work/Life Balance', 'Culture & Values', 'Diversity & Inclusion', 
    'job', 'status'
]

# Fill null values with mode for categorical columns and median for numerical columns
def fill_nulls(df, features):
    for column in features:
        if df[column].dtype == 'object':
            df[column] = df[column].fillna(df[column].mode()[0])
        else:
            df[column] = df[column].fillna(df[column].median())
    return df

df_sampled = fill_nulls(df_sampled, relevant_features + ['rating'])  # Apply to relevant features and 'rating'

# Check for any remaining null values in relevant features
df_sampled[relevant_features + ['rating']].isnull().sum()


pros                         0
cons                         0
Recommend                    0
Career Opportunities         0
Compensation and Benefits    0
Senior Management            0
Work/Life Balance            0
Culture & Values             0
Diversity & Inclusion        0
job                          0
status                       0
rating                       0
dtype: int64

In [3]:
# Frequency encoding for categorical variables
def frequency_encoding(df, columns):
    df_copy = df.copy()  # Create a copy of the DataFrame to avoid the warning
    for column in columns:
        # Generate the frequency encoding (i.e., percentage frequency of each unique value)
        freq = df_copy[column].value_counts() / len(df_copy)
        # Map the original column with its frequency encoding
        df_copy[column] = df_copy[column].map(freq).astype('float32')  # Explicitly cast to float32 to avoid dtype issues
    return df_copy

# Apply frequency encoding to the relevant non-numeric columns
df_sampled = frequency_encoding(df_sampled, relevant_features)

# Downcast again to save memory
df_sampled = downcast(df_sampled)


In [4]:
# Define X and y
X = df_sampled[relevant_features]  # Use only relevant features for X
y = df_sampled['rating']  # Target variable

# Check for non-numeric columns in X
non_numeric_columns_in_X = X.select_dtypes(include=['object', 'category']).columns
print("Non-numeric columns in X:", non_numeric_columns_in_X)

# Print the shapes of X and y
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Non-numeric columns in X: Index([], dtype='object')
Shape of X: (9901889, 11)
Shape of y: (9901889,)


In [5]:
from sklearn.model_selection import train_test_split

# Function to sample data
def sample_data(X, y, sample_size):
    if isinstance(sample_size, float):
        if 0 < sample_size < 1.0:
            return train_test_split(X, y, test_size=0.2, train_size=sample_size, random_state=42)
        elif sample_size == 1.0:
            return train_test_split(X, y, test_size=0.2, train_size=None, random_state=42)  # Fix for 1.0 sample size
        else:
            raise ValueError("sample_size as float must be in the range (0.0, 1.0) or equal to 1.0.")
    elif isinstance(sample_size, int):
        if sample_size > len(X):
            raise ValueError(f"sample_size {sample_size} exceeds the number of available samples {len(X)}.")
        sampled_X = X.sample(n=sample_size, random_state=42)
        sampled_y = y.loc[sampled_X.index]
        return train_test_split(sampled_X, sampled_y, test_size=0.2, random_state=42)
    else:
        raise ValueError("sample_size must be a float or an integer")


In [7]:
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np
import time
import psutil

# Function to calculate and return metrics for LightGBM
def calculate_metrics(X_train, X_test, y_train, y_test):
    # Initialize LightGBM Regressor
    lgbm = lgb.LGBMRegressor(random_state=42)

    # Define hyperparameters for RandomizedSearchCV
    param_distributions = {
        'n_estimators': [50, 100],  # Number of boosting rounds
        'learning_rate': [0.01, 0.05, 0.1],  # Learning rate
        'max_depth': [10, 20, 30],  # Maximum depth, 
        'num_leaves': [31, 40, 50],  # Maximum number of leaves in one tree
        'min_child_samples': [5, 10, 20],  # Minimum number of samples in one leaf
        'subsample': [0.8, 0.9, 1.0],  # Fraction of data to be used for each tree
        'colsample_bytree': [0.8, 0.9, 1.0],  # Fraction of features used for each tree
    }

    random_search = RandomizedSearchCV(lgbm, param_distributions, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)

    start_time = time.time()
    start_cpu = psutil.cpu_percent(interval=None)
    random_search.fit(X_train, y_train)
    end_cpu = psutil.cpu_percent(interval=None)
    end_time = time.time()

    # Calculate time and CPU usage
    execution_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    # Make predictions
    y_pred = random_search.predict(X_test)
    
    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Calculate normalized RMSE (nRMSE)
    target_range = y_train.max() - y_train.min()
    nrmse = rmse / target_range

    # Memory usage
    memory_usage_MB = X_train.memory_usage(deep=True).sum() / (1024 ** 2)
    normalized_time = execution_time / memory_usage_MB

    return {
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'nRMSE': nrmse,  # Normalized RMSE
        'Execution Time (Raw)': execution_time,  # Raw execution time
        'Normalized Time (s/MB)': normalized_time,  # Normalized execution time
        'Average CPU Usage': avg_cpu_usage
    }



In [8]:
import gc  # Garbage Collector

# Define sample sizes
sample_sizes = [1.0, 0.5, 0.25, 0.125, 100, 1000, 10000, 100000]  

# Initialize metrics storage
metrics_list = []
total_execution_time = 0
total_cpu_usage = 0
total_memory_usage_MB = 0

# Loop through each sample size
for size in sample_sizes:
    try:
        # Sample data based on the defined sizes
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = sample_data(X, y, size)
        
        # Calculate metrics
        metrics = calculate_metrics(X_train_sample, X_test_sample, y_train_sample, y_test_sample)
        metrics['Sample Size'] = size
        metrics_list.append(metrics)

        # Call garbage collection after each iteration to free up memory
        gc.collect()

        # Accumulate total metrics
        total_execution_time += metrics['Execution Time (Raw)']
        total_cpu_usage += metrics['Average CPU Usage']
        total_memory_usage_MB += X_train_sample.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"Metrics for sample size {size}:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("-" * 50)

    except Exception as e:
        print(f"An error occurred for sample size {size}: {e}")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.283614 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 720
[LightGBM] [Info] Number of data points in the train set: 7921511, number of used features: 11
[LightGBM] [Info] Start training from score 3.557866
Metrics for sample size 1.0:
RMSE: 0.6996663686187731
MAPE: 0.19921380380638473
R2: 0.6865299022965287
nRMSE: 0.17491659215469327
Execution Time (Raw): 1057.4562313556671
Normalized Time (s/MB): 2.6918505273173263
Average CPU Usage: 56.9
Sample Size: 1.0
--------------------------------------------------
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.061128 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, 

In [9]:
# Convert metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate total metrics
total_avg_cpu_usage = total_cpu_usage / len(sample_sizes)
normalized_total_time = total_execution_time / total_memory_usage_MB

# Convert total execution time to minutes and seconds
total_minutes = int(total_execution_time // 60)
total_seconds = total_execution_time % 60

# Display total metrics
print(f"Total Execution Time for Entire Process (Raw): {total_minutes} minutes and {total_seconds:.2f} seconds")
print(f"Total Normalized Execution Time for Entire Process: {normalized_total_time:.8f} seconds per MB")
print(f"Total Average CPU Usage for Entire Process: {total_avg_cpu_usage:.2f}%")

# Display the metrics DataFrame
metrics_df


Total Execution Time for Entire Process (Raw): 37 minutes and 28.45 seconds
Total Normalized Execution Time for Entire Process: 2.71910590 seconds per MB
Total Average CPU Usage for Entire Process: 68.71%


,RMSE,MAPE,R2,nRMSE,Execution Time (Raw),Normalized Time (s/MB),Average CPU Usage,Sample Size
0,0.699666,0.199214,0.686530,0.174917,1057.456231,2.691851,56.90,1.000
1,0.699573,0.199274,0.686614,0.174893,699.327524,2.848323,70.55,0.500
2,0.699843,0.199310,0.686371,0.174961,321.455133,2.618538,74.30,0.250
3,0.700077,0.199503,0.686162,0.175019,146.353752,2.384363,74.65,0.125
4,0.587390,0.166473,0.771505,0.146848,1.008527,254.210938,73.60,100.000
5,0.815260,0.262309,0.593512,0.203815,2.873197,72.422242,65.15,1000.000
6,0.739552,0.209485,0.635168,0.184888,4.469184,11.265095,73.05,10000.000
7,0.712389,0.206089,0.679056,0.178097,15.507784,3.908916,61.50,100000.000


Another iteration

In [10]:
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np
import time
import psutil

# Function to calculate and return metrics for LightGBM Regressor
def calculate_metrics(X_train, X_test, y_train, y_test):
    gbm = lgb.LGBMRegressor(random_state=42)

    # Define hyperparameters for RandomizedSearchCV
    param_distributions = {
        'n_estimators': [50, 75],  # Number of boosting stages
        'learning_rate': [0.05, 0.1],  # Step size shrinkage
        'max_depth': [3, 5, 7],  # Maximum depth of individual trees
        'min_child_samples': [20, 30],  # Minimum number of samples in a leaf
        'subsample': [0.8, 0.9],  # Fraction of samples used for training
        'colsample_bytree': [0.8, 1.0],  # Fraction of features used per tree
        'force_col_wise': [True]
    }

    random_search = RandomizedSearchCV(gbm, param_distributions, n_iter=10, cv=3, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)

    start_time = time.time()
    start_cpu = psutil.cpu_percent(interval=None)
    random_search.fit(X_train, y_train)
    end_cpu = psutil.cpu_percent(interval=None)
    end_time = time.time()

    # Calculate time and CPU usage
    execution_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    y_pred = random_search.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Calculate the range of the target variable
    target_range = y_train.max() - y_train.min()

    # Calculate normalized RMSE (nRMSE)
    nrmse = rmse / target_range
    
    memory_usage_MB = X_train.memory_usage(deep=True).sum() / (1024 ** 2)
    normalized_time = execution_time / memory_usage_MB
    
    return {
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'nRMSE': nrmse,  # Normalized RMSE
        'Execution Time (Raw)': execution_time,  # Raw execution time
        'Normalized Time (s/MB)': normalized_time,  # Normalized execution time
        'Average CPU Usage': avg_cpu_usage
    }


In [11]:
import gc  # Garbage Collector

# Define sample sizes
sample_sizes = [1.0, 0.5, 0.25, 0.125, 100, 1000, 10000, 100000]  

# Initialize metrics storage
metrics_list = []
total_execution_time = 0
total_cpu_usage = 0
total_memory_usage_MB = 0

# Loop through each sample size
for size in sample_sizes:
    try:
        # Sample data based on the defined sizes
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = sample_data(X, y, size)
        
        # Calculate metrics
        metrics = calculate_metrics(X_train_sample, X_test_sample, y_train_sample, y_test_sample)
        metrics['Sample Size'] = size
        metrics_list.append(metrics)

        # Call garbage collection after each iteration to free up memory
        gc.collect()

        # Accumulate total metrics
        total_execution_time += metrics['Execution Time (Raw)']
        total_cpu_usage += metrics['Average CPU Usage']
        total_memory_usage_MB += X_train_sample.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"Metrics for sample size {size}:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("-" * 50)

    except Exception as e:
        print(f"An error occurred for sample size {size}: {e}")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 720
[LightGBM] [Info] Number of data points in the train set: 7921511, number of used features: 11
[LightGBM] [Info] Start training from score 3.557866
Metrics for sample size 1.0:
RMSE: 0.7052148938193478
MAPE: 0.2023657006619458
R2: 0.681538399663183
nRMSE: 0.17630372345483694
Execution Time (Raw): 242.56266832351685
Normalized Time (s/MB): 0.6174652219856696
Average CPU Usage: 50.95
Sample Size: 1.0
--------------------------------------------------
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 727
[LightGBM] [Info] Number of data points in the train set: 4950944, number of used features: 11
[LightGBM] [Info] Start training from score 3.558041
Metrics for sample size 0.5:
RMSE: 0.7050431513252312
MAPE: 0.20223110594055685
R2: 0.681693492047653
nRMSE: 0.1762607878313078
Execution Time (Raw): 143.38859748840332
Norm

In [12]:
# Convert metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate total metrics
total_avg_cpu_usage = total_cpu_usage / len(sample_sizes)
normalized_total_time = total_execution_time / total_memory_usage_MB

# Convert total execution time to minutes and seconds
total_minutes = int(total_execution_time // 60)
total_seconds = total_execution_time % 60

# Display total metrics
print(f"Total Execution Time for Entire Process (Raw): {total_minutes} minutes and {total_seconds:.2f} seconds")
print(f"Total Normalized Execution Time for Entire Process: {normalized_total_time:.8f} seconds per MB")
print(f"Total Average CPU Usage for Entire Process: {total_avg_cpu_usage:.2f}%")

# Display the metrics DataFrame
metrics_df


Total Execution Time for Entire Process (Raw): 8 minutes and 14.60 seconds
Total Normalized Execution Time for Entire Process: 0.59813185 seconds per MB
Total Average CPU Usage for Entire Process: 62.51%


,RMSE,MAPE,R2,nRMSE,Execution Time (Raw),Normalized Time (s/MB),Average CPU Usage,Sample Size
0,0.705215,0.202366,0.681538,0.176304,242.562668,0.617465,50.95,1.000
1,0.705043,0.202231,0.681693,0.176261,143.388597,0.584014,63.00,0.500
2,0.705127,0.202423,0.681618,0.176282,70.135353,0.571315,66.60,0.250
3,0.705379,0.202395,0.681391,0.176345,34.079531,0.555216,64.90,0.125
4,0.809695,0.252665,0.565823,0.202424,0.199569,50.303606,69.85,100.000
5,0.804994,0.247234,0.603684,0.201249,0.438851,11.061755,62.20,1000.000
6,0.739108,0.209755,0.635606,0.184777,0.818280,2.062569,63.40,10000.000
7,0.714302,0.207531,0.677329,0.178576,2.977359,0.750478,59.20,100000.000
